In [0]:
import numpy as np

import tensorflow as tf

tfd = tf.contrib.distributions

Importing data

In [0]:
data_path = "gene_expression.txt"

expression=np.loadtxt(data_path )# observed data
#labels
label = np.loadtxt("labels.txt")

In [8]:


n_y=558   #dim of observed data
n_z=10   #dim of z
learn_rate=0.0001
logW= tf.Variable(np.random.normal(size=n_y))# log of W
loglambda=tf.Variable(np.random.normal(size=1))#log of lambda
print(logW.shape)

(558,)


In [0]:
def get_batches(X, batch_size = 100): # get batches of traing data
    
	
    n_batches =len(X)//batch_size
    X = X[:n_batches*batch_size]

	# Loop over batches and yield
    for b in range(0, len(X), batch_size):
        yield X[b:b+batch_size]

In [0]:
#encorder
def encoder(x):
    mu = tf.layers.dense(x, n_z)
    logsigma = tf.layers.dense(x, n_z)
    return mu, logsigma


#decoder
def decoder(z):
    f_z=tf.layers.dense(z, n_y)
    return f_z

In [11]:
data= tf.placeholder(tf.float32, [None, n_y], name = 'y')

#z posterior sampling
def sampling_z( mu, logsigma):
    std = tf.exp(0.5*logsigma)
    eps = tf.random_normal(shape=[tf.shape(data)[0],
                              n_z],
                       mean=0.0,
                       stddev=1.0)
    return mu + tf.sqrt(std) * eps
# posterior of z given y
z_mu, z_logsigma = encoder(data)
print(z_logsigma.shape)
z = sampling_z(z_mu, z_logsigma)
print(z)
x_tilde = decoder(z)
W = tf.exp(logW)
lam_bda=tf.exp(loglambda)

(?, 10)
Tensor("add_1:0", shape=(?, 10), dtype=float32)


In [0]:
W=tf.cast(W,tf.float32)
lam_bda=tf.cast(lam_bda,tf.float32)
er=1e-8
l_zero =  -tf.log(1+2*W*lam_bda)/2 - x_tilde*x_tilde*lam_bda/(1+2*W*lam_bda)
print(l_zero.shape)
l_nonzero=tf.log(1-tf.exp(-lam_bda*data*data)+er)  - tf.log(W+er)/2 - (data-x_tilde)*(data-x_tilde)/(2*W)
l=tf.multiply(tf.cast(data<er,tf.float32),l_zero)+tf.multiply(tf.cast(data>=er,tf.float32),l_nonzero)
Y = tf.distributions.Normal(tf.zeros_like(z_mu), tf.ones_like(z_logsigma))
X = tf.distributions.Normal(z_mu, tf.sqrt(tf.exp(z_logsigma)))
     
kl_z =tf.distributions.kl_divergence(X, Y)
print(kl_z)     
lossv = - tf.reduce_sum(l,1) +tf.reduce_sum(kl_z,1) 
loss=(tf.reduce_mean(lossv))
train_op = tf.train.AdamOptimizer(learn_rate).minimize(loss)

In [0]:

sess=tf.Session()
iteration=1
sess.run(tf.global_variables_initializer())
los_old=0
for i in range(800):
   for X in get_batches(expression, 100):#batch_size=100
           
                sess.run(train_op,{data: X})
                iteration=iteration+1
    
            
            # Print at each 5 iters
                if (iteration % 10 == 0):
                  los=sess.run(loss,{data: expression})
                  print(los)
                
                  if abs(los-los_old)<0.05:
                    break
                  los_old=los
  

In [0]:
Z=sess.run(z,{data: expression})
sess.close()

In [21]:

from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import adjusted_rand_score as ARI

labels_pred_Z = KMeans(7, n_jobs=8, n_init=100).fit_predict(Z)
print("Tthe NMI and ARI is")
[NMI(label, labels_pred_Z), ARI(label, labels_pred_Z)]

Tthe NMI and ARI is


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


[0.7540025500601544, 0.7114940572618722]